***Vorlesung 'Syntax natürlicher Sprachen', WS 2020/21***

---
# Übung 11 (Lösung)

In [1]:
import nltk
from collections import defaultdict
from questions import aufgabe
from exercises_11 import *

---
## Aufgabe 1: Weiterverarbeitung syntaktischer Analysen

#### In dieser Aufgabe sollen Sie die Ausgaben eines state-of-the-art-Parsers, nämlich des spacy-Parsers, weiterverarbeiten.

#### Mit dem Ziel, Sie erst einmal mit den typischen Strukturen einer solchen Aufgabe vertraut zu machen, sollen Sie in dieser Aufgabe lediglich entscheiden, ob die Eingabe einen Infinitivsatz mit Akkusativobjekt enthält.

#### Zur Klarheit betrachten Sie die folgenden positiven und negativen Beispiele:

In [2]:
pos_examples = [
    "Er beabsichtigt , den Kuchen ganz alleine zu essen .",
    "Er behauptet , ihn gesehen zu haben ."
]
neg_examples = [
    "Er glaubt , nach Hause zu fliegen .",
    "Zu fliegen ist schön .",
    "Er will gehen ."
]

#### Zur Erinnerung die wichtigsten Schritte zur Nutzung von spacy:

    1. Modell laden

In [3]:
import spacy
nlp = spacy.load('de_core_news_sm')

    2. Parsen und Visualisieren

In [4]:
for sentence in pos_examples + neg_examples:
    analyzed = nlp(sentence)
    spacy.displacy.render(analyzed)

    3. Labels nachschlagen

In [5]:
spacy.explain('oc')

'clausal object'

#### Betrachten Sie die Ausgabe für die Beispielsätze. Schreiben Sie dann eine Funktion, die `True` zurückgibt, wenn ein Satz mit einem Infinitv, der ein Objekt hat, vorliegt und `False` sonst.

In [6]:
def find_accusative(subtree):
    # Hilfsfunktion, die rekursiv Akkusativobjekte sucht
    for child in subtree.children:
        if child.dep_ == 'oa':
            return True
        if child.dep_ == 'oc':
            if find_accusative(child):
                return True
    return False

def classify(sentence):
    # Satz parsen (Syntaxbaum generieren)
    analyzed = nlp(sentence)
    
    # Nach clausal object suchen
    for token in analyzed:
        if token.dep_ == 'oc':
            # wenn gefunden, suche nach Akkusativobjekt
            if find_accusative(token):
                return True

    # wenn Infinitiv in keiner VP nicht gefunden: return False
    return False

Die Ausgabe sollte sein:

```
True
True
False
False
False
```

In [7]:
for p in pos_examples:
    print(classify(p))
for n in neg_examples:
    print(classify(n))

True
True
False
False
False


---
## Aufgabe 2: Informationsextraktion per Syntaxanalyse

#### Gegenstand dieser Aufgabe ist eine anwendungsnahe Möglichkeit, Ergebnisse einer Syntaxanalyse weiterzuverarbeiten. Aus den syntaktischen Abhängigkeiten eines Textes soll (unter Zuhilfenahme einiger Normalisierungsschritte) eine semantische Repräsentation der im Text enthaltenen Informationen gewonnen werden.

#### Für die syntaktische Analyse soll wieder der Dependency Parser von spacy verwendet werden. Die semantische Repräsentation einer Aussage sei ein <a href="https://de.wikipedia.org/wiki/Ontologie_(Informatik)">Knowledge Graph</a> Tripel bestehend aus Subjekt, Prädikat und Objekt (Bei Fehlen von Subjekt oder Objekt soll `None` geschrieben werden.). Die Menge der Prädikate sei durch die Lemmata der vorkommenden Verben definiert. Sie können bei der Implementierung davon ausgehen, dass kein Satz zwei verschiedene Aussagen mit dem gleichen Prädikat enthält.

#### Folgendes Beispiel illustriert das gewünschte Ergebnis:

#### Eingabe:

    I shot an elephant in my pajamas.
    The elephant was seen by a giraffe in the desert.
    The bird I need is a raven.
    The man who saw the raven laughed out loud.

#### Ausgabe:

    (I, shoot, elephant)
    (giraffe, see, elephant)
    (I, need, bird)
    (bird, IS, raven)
    (man, laugh, None)
    (man, see, raven)

In [8]:
sentences = [
    "I shot an elephant in my pajamas.",
    "The elephant was seen by a giraffe in the desert.",
    "The bird I need is a raven.",
    "The man who saw the raven laughed out loud.",
]

In [9]:
nlp = spacy.load('en_core_web_sm')

In [10]:
spacy.displacy.render(nlp(sentences[0]))

In [11]:
for token in nlp(sentences[0]):
    print(token.dep_, token.text, token.lemma_)

nsubj I -PRON-
ROOT shot shoot
det an an
dobj elephant elephant
prep in in
poss my -PRON-
pobj pajamas pajama
punct . .


In [12]:
def is_overwritable(token):
    return token is None or token.text in ['who', 'which', 'that', 'whom']

def generate_predicates_for_dep(analyzed):
    predicates = defaultdict(lambda: [None, None])
    
    for token in analyzed:
        if token.dep_ == 'nsubj' and token.head.lemma_ != 'be':
            args = predicates[token.head.lemma_]
            if is_overwritable(args[0]):
                args[0] = token

        if token.dep_ == 'dobj' or token.dep_ == 'nsubjpass':
            args = predicates[token.head.lemma_]
            if is_overwritable(args[1]):
                args[1] = token
        
        if token.dep_ == 'agent':
            for child in token.children:
                if child.dep_ == 'pobj':
                    predicates[token.head.lemma_][0] = child
        
        if token.dep_ == 'attr':
            for sibling in token.head.children:
                if sibling.dep_ == 'nsubj':
                    predicates["IS"][0] = sibling
                    predicates["IS"][1] = token.lemma_
        
        if token.dep_ == 'relcl':
            args = predicates[token.lemma_]
            if is_overwritable(args[0]):
                args[0] = token.head
            elif is_overwritable(args[1]):
                args[1] = token.head
            
    return predicates

def generate_predicates_for_sentence(sentence):    
    analyzed = nlp(sentence)
    predicates = generate_predicates_for_dep(analyzed)
                    
    return [
        "({}, {}, {})".format(elements[0], pred, elements[1])
        for pred, elements in predicates.items()
    ]

In [13]:
for pred in generate_predicates_for_sentence(sentences[3]):
    print(pred)

(man, laugh, None)
(man, see, raven)


In [14]:
def generate_predicates_for_text(text):
    predicates = []
    for sent in text:
        predicates.extend(generate_predicates_for_sentence(sent))
    return predicates

In [15]:
for pred in generate_predicates_for_text(sentences):
    print(pred)

(I, shoot, elephant)
(giraffe, see, elephant)
(I, need, bird)
(bird, IS, raven)
(man, laugh, None)
(man, see, raven)


#### Ideale Ausgabe:

    (I, shoot, elephant)
    (giraffe, see, elephant)
    (I, need, bird)
    (bird, IS, raven)
    (man, laugh, None)
    (man, see, raven)

---
# Hausaufgaben

---
## Aufgabe 3: Parent Annotation

#### *Parent Annotation* kann die Performanz einer CFG wesentlich verbessern. Schreiben Sie eine Funktion, die einen gegebenen Syntaxbaum dieser Optimierung unterzieht. Auf diese Art und Weise transformierte Bäume können dann wiederum zur Grammatikinduktion verwendet werden.

#### `parentHistory` soll dabei die Anzahl der Vorgänger sein, die zusätzlich zum direkten Elternknoten berücksichtigt werden. (Die Berücksichtigung dieses Parameters ist optional.)

#### `parentChar` soll ein Trennzeichen sein, das bei den neuen Knotenlabels zwischen dem ursprünglichen Knotenlabel und der Liste von Vorgängern eingefügt wird.

In [16]:
def parent_annotation(tree, parentHistory=0, parentChar="^"):
    def pa_rec(node, parents):
        originalNode = node.label() 
        parentString = (
            parentChar + '<' + '-'.join(parents) + '>'  
        )
        node.set_label(node.label() + parentString) 
        for child in node: 
            pa_rec(
                child,
                [originalNode] + parents[:parentHistory] 
            )
        return node
    return pa_rec(tree, []) 

In [17]:
test_tree = nltk.Tree(
    "S",
    [
        nltk.Tree("NP", [
            nltk.Tree("DET", []),
            nltk.Tree("N", [])
        ]),
        nltk.Tree("VP", [
            nltk.Tree("V", []),
            nltk.Tree("NP", [
                nltk.Tree("DET", []),
                nltk.Tree("N", [])
            ])
        ])
    ]
)

parent_annotation(test_tree).pretty_print()

                        S^<>                         
            _____________|_______                     
           |                   VP^<S>                
           |              _______|________            
         NP^<S>          |             NP^<VP>       
    _______|______       |        ________|______     
DET^<NP>        N^<NP> V^<VP> DET^<NP>         N^<NP>
   |              |      |       |               |    
  ...            ...    ...     ...             ...  



---
## Aufgabe 4: Mehr Semantik für IE

#### Zusätzlich zu den in Aufgabe 2 behandelten Konstruktionen sollen jetzt auch negierte und komplexe Sätze mit Konjunktionen sinnvoll verarbeitet werden.

#### Eingabe:

    I see an elephant.
    You didn't see the elephant.
    Peter saw the elephant and drank wine.
    
#### Gewünschte Ausgabe:

    (I, see, elephant)
    (You, not_see, elephant)
    (Peter, see, elephant)
    (Peter, drink, wine)
    
#### Kopieren Sie am besten Ihren aktuellen Stand von oben herunter und fügen Sie Ihre Erweiterungen dann hier ein.    

In [18]:
sentences = [
    "I see an elephant.",
    "You didn't see the elephant.",
    "Peter saw the elephant and drank wine."
]

In [19]:
def is_overwritable(token):
    return token is None or token.text in ['who', 'which', 'that', 'whom']

def generate_predicates_for_dep(analyzed):
    predicates = defaultdict(lambda: [None, None])
    negated = set()
    conj = defaultdict(set)
    
    for token in analyzed:
        if token.dep_ == 'nsubj' and token.head.lemma_ != 'be':
            args = predicates[token.head.lemma_]
            if is_overwritable(args[0]):
                args[0] = token
        
        if token.dep_ == 'dobj' or token.dep_ == 'nsubjpass':
            args = predicates[token.head.lemma_]
            if is_overwritable(args[1]):
                args[1] = token
        
        if token.dep_ == 'agent':
            for child in token.children:
                if child.dep_ == 'pobj':
                    predicates[token.head.lemma_][0] = child
        
        if token.dep_ == 'attr':
            for sibling in token.head.children:
                if sibling.dep_ == 'nsubj':
                    predicates["IS-A"][0] = sibling
                    predicates["IS-A"][1] = token.lemma_
        
        if token.dep_ == 'relcl':
            args = predicates[token.lemma_]
            if is_overwritable(args[0]):
                args[0] = token.head
            elif is_overwritable(args[1]):
                args[1] = token.head
                
        if token.dep_ == 'neg':
            negated.add(token.head.lemma_)
            
        if token.dep_ == 'conj':
            conj[token.lemma_].add(token.head.lemma_)
            
    for cc1 in conj:
        for cc2 in conj[cc1]:
            if cc2 in predicates:
                args = predicates[cc1]
                cc_args = predicates[cc2]
                if is_overwritable(args[0]):
                    args[0] = cc_args[0]
                if is_overwritable(args[1]):
                    args[1] = cc_args[1]
            
    for neg in negated:
        predicates['not_' + neg] = predicates[neg]
        del predicates[neg]
            
    return predicates

def generate_predicates_for_sentence(sentence):    
    analyzed = nlp(sentence)
    predicates = generate_predicates_for_dep(analyzed)
                    
    return [
        "({}, {}, {})".format(elements[0], pred, elements[1])
        for pred, elements in predicates.items()
    ]

In [20]:
for pred in generate_predicates_for_text(sentences):
    print(pred)

(I, see, elephant)
(You, not_see, elephant)
(Peter, see, elephant)
(Peter, drink, wine)


#### Ideale Ausgabe:

    (I, see, elephant)
    (You, not_see, elephant)
    (Peter, see, elephant)
    (Peter, drink, wine)

---
## Aufgabe 5: Fragen zu NLTK-08-extras, 2.9 ("Viterbi-Parser")

#### 1. Betrachten Sie die folgende im Kapitel gegebene PCFG:

```
grammar = nltk.PCFG.fromstring('''
  NP  -> NNS [0.5] | JJ NNS [0.3] | NP CC NP [0.2]
  NNS -> "cats" [0.1] | "dogs" [0.2] | "mice" [0.3] | NNS CC NNS [0.4]
  JJ  -> "big" [0.4] | "small" [0.6]
  CC  -> "and" [0.9] | "or" [0.1]
  ''')
viterbi_parser = nltk.ViterbiParser(grammar)

sent = 'big cats and dogs'.split()
for tree in viterbi_parser.parse(sent):
    print(tree)
```

`(NP (JJ big) (NNS (NNS cats) (CC and) (NNS dogs))) (p=***)`

In [21]:
aufgabe(blatt11_5_1)

OpenQuestion(children=(HTML(value='<h4 style="font-size:14px;">Berechnen Sie die Wahrscheinlichkeit für die Ab…

---
#### 2. Betrachten Sie das dazugehörige Tracing-Output des `ViterbiParser`s:

```
>>> sent = 'big cats and dogs'.split()
>>> viterbi_parser = nltk.ViterbiParser(grammar, trace = 3)
>>> for tree in viterbi_parser.parse(sent):
>>>     print(tree)

Inserting tokens into the most likely constituents table...
   Insert: |=...| big
   Insert: |.=..| cats
   Insert: |..=.| and
   Insert: |...=| dogs
Finding the most likely constituents spanning 1 text elements...
   Insert: |=...| JJ -> 'big' [0.4]                 0.4000000000
   Insert: |.=..| NNS -> 'cats' [0.1]               0.1000000000
   Insert: |.=..| NP -> NNS [0.5]                   0.0500000000
   Insert: |..=.| CC -> 'and' [0.9]                 0.9000000000
   Insert: |...=| NNS -> 'dogs' [0.2]               0.2000000000
   Insert: |...=| NP -> NNS [0.5]                   0.1000000000
Finding the most likely constituents spanning 2 text elements...
   Insert: |==..| NP -> JJ NNS [0.3]                0.0120000000
Finding the most likely constituents spanning 3 text elements...
   Insert: |.===| NP -> NP CC NP [0.2]              0.0009000000
   Insert: |.===| NNS -> NNS CC NNS [0.4]           0.0072000000
   Insert: |.===| NP -> NNS [0.5]                   0.0036000000
  Discard: |.===| NP -> NP CC NP [0.2]              0.0009000000
  Discard: |.===| NP -> NP CC NP [0.2]              0.0009000000
Finding the most likely constituents spanning 4 text elements...
   Insert: |====| NP -> JJ NNS [0.3]                0.0008640000
  Discard: |====| NP -> NP CC NP [0.2]              0.0002160000
  Discard: |====| NP -> NP CC NP [0.2]              0.0002160000
(NP (JJ big) (NNS (NNS cats) (CC and) (NNS dogs))) (p=***)
```

In [22]:
aufgabe(blatt11_5_2)

MultipleChoice(children=(HTML(value='<h4 style="font-size:14px;">Warum werden die Analysen in den Discard-Zeil…

---
#### 3. Ein statistischer ChartParser findet folgende 2 Ableitungen für die NP *big cats and dogs*:
```
(NP (JJ big) (NNS (NNS cats) (CC and) (NNS dogs))) (p=0.000864)
(NP (NP (JJ big) (NNS cats)) (CC and) (NP (NNS dogs))) (p=0.000216)
```

In [23]:
# Visualisierung
from nltk import Tree

tree1 = Tree.fromstring("(NP (JJ big) (NNS (NNS cats) (CC and) (NNS dogs)))")
tree2 = Tree.fromstring("(NP (NP (JJ big) (NNS cats)) (CC and) (NP (NNS dogs)))")

tree1.pretty_print(unicodelines=True)
tree2.pretty_print(unicodelines=True)

          NP         
 ┌────────┴───┐       
 │           NNS     
 │   ┌────────┼───┐   
 JJ NNS       CC NNS 
 │   │        │   │   
big cats     and dogs

         NP          
     ┌───┴────┬───┐   
     NP       │   NP 
 ┌───┴───┐    │   │   
 JJ     NNS   CC NNS 
 │       │    │   │   
big     cats and dogs



In [24]:
aufgabe(blatt11_5_3)

SingleChoice(children=(HTML(value='<h4 style="font-size:14px;">Um welchen Parser kann es sich nicht handeln?</…

SingleChoice(children=(HTML(value='<h4 style="font-size:14px;">Nach welchem Kriterium wird beim Parsen mit dem…

SingleChoice(children=(HTML(value='<h4 style="font-size:14px;">Um welche Art der Ambiguität handelt es sich be…